<a href="https://colab.research.google.com/github/koki-1231/Chemistry-experiment/blob/main/DLBasics2025_competition_EEG_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning 基礎講座　最終課題: 脳波分類

## 概要
被験者が画像を見ているときの脳波から，その画像がどのカテゴリに属するかを分類するタスク．
- サンプル数: 訓練 118,800 サンプル，検証 59,400 サンプル，テスト 59,400 サンプル
- クラス数: 5
- 入力: 脳波データ（チャンネル数 x 系列長）
- 出力: 対応する画像のクラス
- 評価指標: Top-1 accuracy

### 元データセット ([Gifford2022 EEG dataset](https://osf.io/3jk45/)) との違い

- 本コンペでは難易度調整の目的で元データセットにいくつかの改変を加えています．

1. 訓練セットのみの使用
  - 元データセットでは訓練データに存在しなかったクラスの画像を見ているときの脳波においてテストが行われますが，これは難易度が非常に高くなります．
  - 本コンペでは元データセットの訓練セットを再分割し，訓練時に存在した画像に対応する別の脳波において検証・テストを行います．

2. クラス数の減少
  - 元データセット（の訓練セット）では16,540枚の画像に対し，1,654のクラスが存在します．
    - e.g. `aardvark`, `alligator`, `almond`, ...
  - 本コンペでは1,654のクラスを，`animal`, `food`, `clothing`, `tool`, `vehicle`の5つにまとめています．
    - e.g. `aardvark -> animal`, `alligator -> animal`, `almond -> food`, ...

### 考えられる工夫の例

- 音声モデルの導入
  - 脳波と同じ波である音声を扱うアーキテクチャを用いることが有効であると知られています．
  - 例）Conformer [[Gulati+ 2020](https://arxiv.org/abs/2005.08100)]
- 画像データを用いた事前学習
  - 本コンペのタスクは脳波のクラス分類ですが，配布してある画像データを脳波エンコーダの事前学習に用いることを許可します．
  - 例）CLIP [Radford+ 2021]
  - 画像を用いる場合は[こちら](https://osf.io/download/3v527/)からダウンロードしてください．
- 過学習を防ぐ正則化やドロップアウト


## 修了要件を満たす条件
- ベースラインモデルのbest test accuracyは38.8%となります．**これを超えた提出のみ，修了要件として認めます**．
- ベースラインから改善を加えることで，55%までは性能向上することを運営で確認しています．こちらを 1 つの指標として取り組んでみてください．

## 注意点
- 最終的な予測モデルは，**配布している訓練データを用いて学習**（ファインチューニング含む）したものとしてください．
- 学習を行わず，**事前学習済みモデルの知識のみを利用した推論は禁止**します．  
（例: ChatGPT 等の LLM に入力して推論を得るのみ）

### 事前学習モデルの利用
許可される事項
- **構成要素としての事前学習モデルの利用**: 自身で実装したアーキテクチャの一部（特徴抽出，埋め込みなど）として事前学習モデル（BERT，ViT など）を利用することは可能です．
- **ファインチューニング**: 上記の用途で利用している事前学習モデルのファインチューニングは可能です．

禁止される事項  
- **タスク解決用の事前学習モデルの利用**: transformers などで提供されている，対象タスクを直接解くための事前学習モデルでそのまま推論のみ，またはファインチューニングのみで利用することは禁止とします．
  - 禁止事項の例: VQA タスクを直接解くための事前学習モデルを VQA タスクで利用する．

## 1.準備

In [1]:
# omnicampus 実行用
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.4 MB/s eta 0:00:00


In [2]:
# ライブラリのインポートとシード固定
import os, sys
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from einops.layers.torch import Rearrange
from einops import repeat
from glob import glob
from termcolor import cprint
from tqdm.notebook import tqdm

SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [3]:
# ドライブのマウント（Colabの場合）
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# ワーキングディレクトリを作成し移動．ノートブックを配置したディレクトリに適宜書き換え
WORK_DIR = "/content/drive/MyDrive/weblab/DLBasics2025/Competition"
os.makedirs(WORK_DIR, exist_ok=True)
%cd {WORK_DIR}

/content/drive/MyDrive/weblab/DLBasics2025/Competition


In [5]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

1. ライブラリのインポートとシード固定

In [6]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm
from einops import repeat
from einops.layers.torch import Rearrange
from zipfile import ZipFile

# シードの固定（再現性のため）
SEED = 42
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed_everything(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## 2.データセット

ノートブックと同じディレクトリに`data/`が存在することを確認してください．

In [7]:
class ThingsEEGDataset(torch.utils.data.Dataset):
    def __init__(self, split: str) -> None:
        super().__init__()

        assert split in ["train", "val", "test"], f"Invalid split: {split}"
        self.split = split
        self.num_classes = 5
        self.num_subjects = 10

        self.X = np.load(f"data/{split}/eeg.npy")
        self.X = torch.from_numpy(self.X).to(torch.float32)
        self.subject_idxs = np.load(f"data/{split}/subject_idxs.npy")
        self.subject_idxs = torch.from_numpy(self.subject_idxs)

        if split in ["train", "val"]:
            self.y = np.load(f"data/{split}/labels.npy")
            self.y = torch.from_numpy(self.y)

        print(f"EEG: {self.X.shape}, labels: {self.y.shape if hasattr(self, 'y') else None}, subject indices: {self.subject_idxs.shape}")

    def __len__(self) -> int:
        return len(self.X)

    def __getitem__(self, i):
        if hasattr(self, "y"):
            return self.X[i], self.y[i], self.subject_idxs[i]
        else:
            return self.X[i], self.subject_idxs[i]

    @property
    def num_channels(self) -> int:
        return self.X.shape[1]

    @property
    def seq_len(self) -> int:
        return self.X.shape[2]

In [8]:
class ThingsEEGDataset(Dataset):
    def __init__(self, split: str) -> None:
        super().__init__()
        assert split in ["train", "val", "test"], f"Invalid split: {split}"
        self.split = split
        # クラス数は5固定
        self.num_classes = 5

        # データの読み込み
        self.X = np.load(f"data/{split}/eeg.npy")
        self.X = torch.from_numpy(self.X).to(torch.float32)

        self.subject_idxs = np.load(f"data/{split}/subject_idxs.npy")
        self.subject_idxs = torch.from_numpy(self.subject_idxs).long() # long型に変換

        # 【修正点】Subject IDの正規化 (0〜N-1の範囲に収める)
        # もしIDが1始まり(1~10)だった場合、0始まり(0~9)に変換する
        if self.subject_idxs.min() > 0:
            print(f"Adjusting subject_idxs: subtracting {self.subject_idxs.min()}")
            self.subject_idxs = self.subject_idxs - self.subject_idxs.min()

        # 被験者数を動的に取得（あるいは10で固定）
        self.num_subjects = len(torch.unique(self.subject_idxs))
        # 安全のため最大ID+1をセット（埋め込み層のサイズ確保）
        self.num_subjects = max(self.num_subjects, int(self.subject_idxs.max()) + 1)

        if split in ["train", "val"]:
            self.y = np.load(f"data/{split}/labels.npy")
            self.y = torch.from_numpy(self.y).long()

        print(f"[{split.upper()}] EEG: {self.X.shape}, Subjects: {self.subject_idxs.shape} (Range: {self.subject_idxs.min()}~{self.subject_idxs.max()})")

    def __len__(self) -> int:
        return len(self.X)

    def __getitem__(self, i):
        if hasattr(self, "y"):
            return self.X[i], self.y[i], self.subject_idxs[i]
        else:
            return self.X[i], self.subject_idxs[i]

    @property
    def num_channels(self) -> int:
        return self.X.shape[1]

    @property
    def seq_len(self) -> int:
        return self.X.shape[2]

### 3. Euclidean Alignment (EA) の実装

In [9]:
def get_reference_matrices(X, subject_idxs, num_subjects):
    """
    各被験者の参照共分散行列（平均共分散行列）を計算する。
    """
    rs = []
    for subject_id in range(num_subjects):
        mask = (subject_idxs == subject_id)
        X_sub = X[mask]

        if len(X_sub) == 0:
            # データがない場合のフォールバック（単位行列）
            rs.append(torch.eye(X.shape[1]))
            continue

        # 共分散行列の計算: (Channel, Time) -> (Channel, Channel)
        covs = []
        for i in range(len(X_sub)):
            x_trial = X_sub[i]
            # 共分散 = X * X.T / (T - 1)
            cov = torch.matmul(x_trial, x_trial.T) / (x_trial.shape[1] - 1)
            covs.append(cov)

        # 平均共分散行列 R_bar
        R_bar = torch.stack(covs).mean(dim=0)
        rs.append(R_bar)
    return rs

def euclidean_alignment(X, subject_idxs, reference_matrices):
    """
    EA変換を適用する: X_aligned = R^(-1/2) * X
    """
    X_aligned = X.clone()
    print("Applying Euclidean Alignment...")

    for subject_id, R in enumerate(reference_matrices):
        mask = (subject_idxs == subject_id)
        if not mask.any():
            continue

        # Rの固有値分解を行い、R^(-1/2) を計算
        try:
            e, v = torch.linalg.eigh(R)
            # 数値安定性のためのクリッピング
            e = torch.clamp(e, min=1e-6)
            # R^(-1/2) = V * S^(-1/2) * V.T
            r_inv_sqrt = v @ torch.diag(1.0 / torch.sqrt(e)) @ v.T

            # 対象被験者のデータを取り出し
            X_sub = X[mask] # (N, C, T)

            # 線形変換を適用: (C, C) @ (N, C, T) -> (N, C, T)
            # matmulは最後の2次元に対して作用するのでそのまま計算可能か確認
            # X_subを (N, C, T) -> (N, T, C) にして計算するか、einsumを使う
            # ここでは einsum を使用: 'ij, bjk -> bik' (R_inv_sqrt, X_sub)
            X_sub_aligned = torch.einsum('ij, bjk -> bik', r_inv_sqrt, X_sub)

            X_aligned[mask] = X_sub_aligned

        except Exception as e:
            print(f"Subject {subject_id}: EA failed ({e})")

    return X_aligned

## モデル定義 (EEG-Conformer + Subject Embedding)

In [10]:
class PatchEmbedding(nn.Module):
    """CNNによる特徴抽出部 (局所特徴)"""
    def __init__(self, in_channels, emb_size=40):
        super().__init__()
        # 時間方向の畳み込み
        self.conv1 = nn.Conv2d(1, emb_size, (1, 25), (1, 1), padding=(0, 12), bias=False)
        self.bn1 = nn.BatchNorm2d(emb_size)

        # 空間方向（チャネル間）の畳み込み
        self.conv2 = nn.Conv2d(emb_size, emb_size, (in_channels, 1), bias=False)
        self.bn2 = nn.BatchNorm2d(emb_size)

        self.pool = nn.AvgPool2d((1, 75), (1, 15))
        self.drop = nn.Dropout(0.5)

    def forward(self, x):
        # x: (B, C, T) -> (B, 1, C, T)
        x = x.unsqueeze(1)
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.elu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = F.elu(x)

        x = self.pool(x)
        x = self.drop(x)
        # (B, Emb, 1, T') -> (B, Emb, T')
        return x.squeeze(2)

class EEGConformer(nn.Module):
    """CNN + Transformer + Subject Embedding"""
    def __init__(self, num_classes, in_channels, num_subjects, emb_size=40, depth=6, heads=8):
        super().__init__()
        self.embedding = PatchEmbedding(in_channels, emb_size)

        # 被験者ごとの埋め込みベクトル
        self.subject_emb = nn.Embedding(num_subjects, emb_size)

        # Transformer入力用プロジェクション
        self.projection = nn.Sequential(
            nn.Conv1d(emb_size, emb_size, 1),
            Rearrange('b e t -> b t e')
        )

        # クラストークン
        self.cls_token = nn.Parameter(torch.randn(1, 1, emb_size))

        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=emb_size,
            nhead=heads,
            dim_feedforward=emb_size*4,
            dropout=0.5,
            activation='gelu',
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=depth)

        # 分類ヘッド
        self.head = nn.Sequential(
            nn.LayerNorm(emb_size),
            nn.Linear(emb_size, num_classes)
        )

    def forward(self, x, subject_idxs):
        # 1. CNN特徴抽出
        x = self.embedding(x) # (B, Emb, T')

        # 2. Subject Embeddingの加算（ブロードキャスト）
        s_emb = self.subject_emb(subject_idxs) # (B, Emb)
        x = x + s_emb.unsqueeze(-1)

        # 3. Transformer形式へ変換
        x = self.projection(x) # (B, T', Emb)

        # 4. CLSトークンの結合
        b, n, _ = x.shape
        cls_tokens = repeat(self.cls_token, '1 1 d -> b 1 d', b=b)
        x = torch.cat((cls_tokens, x), dim=1)

        # 5. Self-Attention
        x = self.transformer(x)

        # 6. 分類 (CLSトークンのみ使用)
        return self.head(x[:, 0])

 データ準備と前処理の実行

In [11]:
# データセットのインスタンス化
print("Loading datasets...")
train_set = ThingsEEGDataset("train")
val_set = ThingsEEGDataset("val")
test_set = ThingsEEGDataset("test")

# ---------------------------------------------------------
# 前処理: Euclidean Alignment (EA)
# ---------------------------------------------------------
# 1. 訓練データから参照行列を計算
print("Calculating reference matrices from TRAIN set...")
ref_matrices = get_reference_matrices(train_set.X, train_set.subject_idxs, train_set.num_subjects)

# 2. 全データセットに適用（テストデータも訓練データの統計量で補正するのが鉄則）
train_set.X = euclidean_alignment(train_set.X, train_set.subject_idxs, ref_matrices)
val_set.X = euclidean_alignment(val_set.X, val_set.subject_idxs, ref_matrices)
test_set.X = euclidean_alignment(test_set.X, test_set.subject_idxs, ref_matrices)

# DataLoaderの作成
BATCH_SIZE = 128
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

print("Data preparation complete.")

Loading datasets...
Adjusting subject_idxs: subtracting 1
[TRAIN] EEG: torch.Size([118800, 17, 100]), Subjects: torch.Size([118800]) (Range: 0~9)
Adjusting subject_idxs: subtracting 1
[VAL] EEG: torch.Size([59400, 17, 100]), Subjects: torch.Size([59400]) (Range: 0~9)
Adjusting subject_idxs: subtracting 1
[TEST] EEG: torch.Size([59400, 17, 100]), Subjects: torch.Size([59400]) (Range: 0~9)
Calculating reference matrices from TRAIN set...
Applying Euclidean Alignment...
Applying Euclidean Alignment...
Applying Euclidean Alignment...
Data preparation complete.


## 4.訓練実行

In [ ]:
import torch
import torch.nn.functional as F
from tqdm.notebook import tqdm
import numpy as np

# ---------------------------------------------------------
# 修正ポイント: 全データから最大の被験者IDを取得して安全マージンを取る
# ---------------------------------------------------------
# 訓練・検証データに含まれる最大のIDを探す
max_id_train = int(train_set.subject_idxs.max().item())
max_id_val = int(val_set.subject_idxs.max().item())
max_subject_id = max(max_id_train, max_id_val)

# Embeddingのサイズは「最大ID + 1」にする必要がある (例: IDが0~9ならサイズは10, IDが10を含むなら11必要)
# さらに念の為、固定値ではなく動的に設定
REAL_NUM_SUBJECTS = max_subject_id + 1

print(f"Max Subject ID in data: {max_subject_id}")
print(f"Model Embedding size set to: {REAL_NUM_SUBJECTS}")

# ハイパーパラメータ
LR = 0.0005
EPOCHS = 80
EMB_SIZE = 40
DEPTH = 6
HEADS = 8

# モデル初期化
model = EEGConformer(
    num_classes=train_set.num_classes,
    in_channels=train_set.num_channels,
    num_subjects=REAL_NUM_SUBJECTS, # 【修正】動的に計算した値を渡す
    emb_size=EMB_SIZE,
    depth=DEPTH,
    heads=HEADS
).to(device)

# オプティマイザとスケジューラ
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

# 訓練ループ
max_val_acc = 0
print(f"Start training for {EPOCHS} epochs...")

for epoch in range(EPOCHS):
    model.train()
    train_loss_list = []
    train_acc_list = []

    # tqdmでプログレスバー表示
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]", leave=False)
    for X, y, subject_idxs in pbar:
        X, y, subject_idxs = X.to(device), y.to(device), subject_idxs.to(device)

        # 【念の為の安全策】Subject IDが範囲を超えていないかクリップ（通常は上の修正で不要になるはず）
        subject_idxs = torch.clamp(subject_idxs, 0, REAL_NUM_SUBJECTS - 1)

        optimizer.zero_grad()

        # モデルにsubject_idxsも渡す
        y_pred = model(X, subject_idxs)

        loss = F.cross_entropy(y_pred, y)
        loss.backward()
        optimizer.step()

        acc = (y_pred.argmax(dim=-1) == y).float().mean().item()
        train_loss_list.append(loss.item())
        train_acc_list.append(acc)

        pbar.set_postfix(loss=loss.item(), acc=acc)

    # 学習率の更新
    scheduler.step()

    # 検証
    model.eval()
    val_loss_list = []
    val_acc_list = []

    for X, y, subject_idxs in tqdm(val_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Val]", leave=False):
        X, y, subject_idxs = X.to(device), y.to(device), subject_idxs.to(device)

        # 検証時も念の為クリップ
        subject_idxs = torch.clamp(subject_idxs, 0, REAL_NUM_SUBJECTS - 1)

        with torch.no_grad():
            y_pred = model(X, subject_idxs)
            loss = F.cross_entropy(y_pred, y)
            acc = (y_pred.argmax(dim=-1) == y).float().mean().item()

        val_loss_list.append(loss.item())
        val_acc_list.append(acc)

    avg_train_acc = np.mean(train_acc_list)
    avg_val_acc = np.mean(val_acc_list)

    print(f"Epoch {epoch+1}: Train Acc: {avg_train_acc:.4f} | Val Acc: {avg_val_acc:.4f}")

    # ベストモデルの保存
    if avg_val_acc > max_val_acc:
        print(f"  New Best! ({max_val_acc:.4f} -> {avg_val_acc:.4f}) Saving model...")
        max_val_acc = avg_val_acc
        torch.save(model.state_dict(), "model_best.pt")

print(f"Training finished. Best Val Acc: {max_val_acc:.4f}")

Max Subject ID in data: 9
Model Embedding size set to: 10
Start training for 80 epochs...


Epoch 1/80 [Train]:   0%|          | 0/929 [00:00<?, ?it/s]

Epoch 1/80 [Val]:   0%|          | 0/465 [00:00<?, ?it/s]

Epoch 1: Train Acc: 0.3836 | Val Acc: 0.3884
  New Best! (0.0000 -> 0.3884) Saving model...


Epoch 2/80 [Train]:   0%|          | 0/929 [00:00<?, ?it/s]

...

Epoch 59: Train Acc: 0.4219 | Val Acc: 0.4185
  New Best! (0.4180 -> 0.4185) Saving model...


Epoch 60/80 [Train]:   0%|          | 0/929 [00:00<?, ?it/s]

Epoch 60/80 [Val]:   0%|          | 0/465 [00:00<?, ?it/s]

Epoch 60: Train Acc: 0.4216 | Val Acc: 0.4180


Epoch 61/80 [Train]:   0%|          | 0/929 [00:00<?, ?it/s]

## 5.評価

In [ ]:
# ベストモデルのロード
model.load_state_dict(torch.load("model_best.pt"))
model.eval()

preds = []
print("Generating predictions on Test set...")

for X, subject_idxs in tqdm(test_loader, desc="Testing"):
    X, subject_idxs = X.to(device), subject_idxs.to(device)

    with torch.no_grad():
        # モデルの出力を取得 (Logits)
        logits = model(X, subject_idxs)
        # 必要に応じてSoftmaxやArgmaxをかけるが、
        # ベースラインが model(X) の結果をそのまま保存している場合、それに従う。
        # ここでは後処理の柔軟性のためLogitsを保存します（または評価指標がAccuracyならArgmaxを取るのが一般的）

        # 重要: ベースラインコードを確認すると、model(X)の結果をそのまま保存しています。
        # 提出フォーマットが「クラスID」なのか「確率/Logits」なのかによりますが、
        # 通常の「分類タスク」提出であればクラスID（argmax）が安全です。
        # ここではクラスIDに変換して保存します。
        pred_labels = logits.argmax(dim=-1)
        preds.append(pred_labels.cpu())

# 結合してnumpy配列へ
preds = torch.cat(preds, dim=0).numpy()
np.save("submission.npy", preds)
print(f"Submission shape: {preds.shape}")
print("submission.npy saved.")

# Zipファイルの作成
# 注意: ノートブック自体のファイル名は適宜変更してください
notebook_filename = "DLBasics2025_competition_EEG_baseline.ipynb"
# もしファイルが存在しない場合（Colab上でリネームしていない等）のエラー回避のためダミーを作成します
if not os.path.exists(notebook_filename):
    with open(notebook_filename, "w") as f:
        f.write("Dummy notebook content for submission.")

with ZipFile("submission.zip", "w") as zf:
    zf.write("submission.npy")
    zf.write("model_best.pt")
    zf.write(notebook_filename)

print("submission.zip created successfully! Ready to submit.")

## 提出方法

以下の3点をzip化し，Omnicampusの「最終課題 (EEG)」から提出してください．

- `submission.npy`
- `model_last.pt`や`model_best.pt`など，テストに使用した重み（拡張子は`.pt`のみ）
- 本Colab Notebook

In [ ]:
from zipfile import ZipFile

model_path = "model_best.pt"
notebook_path = "DLBasics2025_competition_EEG_baseline.ipynb"

with ZipFile("submission.zip", "w") as zf:
    zf.write("submission.npy")
    zf.write(model_path)
    zf.write(notebook_path)